In [1]:
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa

In [2]:
#Leemos el archivo reviews.parquet en un DataFrame
gamesdf = pq.read_table("../0-DATA/output_steam_gamesEDA.parquet").to_pandas()
gamesdf.head(5)

,app_name,price,id,developer,genres_clean
0,Lost Summoner Kitty,4.99,761140.0,Kotoshiro,Action
1,Ironbound,0.00,643980.0,Secret Level SRL,Free to Play
2,Real Pool 3D - Poolians,0.00,670290.0,Poolians.com,Casual
3,弹炸人2222,0.99,767400.0,彼岸领域,Action
4,Battle Royale Trainer,3.99,772540.0,Trickjump Games Ltd,Action


In [3]:
#Leemos el archivo reviews.parquet en un DataFrame
reviewsdf = pq.read_table("../0-DATA/australian_user_reviewsEDA.parquet").to_pandas()
reviewsdf.head(5)

,item_id,recommend,user_id,sentiment_analysis,año
0,1250,1,76561197970982479,2,2011
1,22200,1,76561197970982479,1,2011
2,43110,1,76561197970982479,2,2011
3,251610,1,js41637,2,2014
4,227300,1,js41637,2,2013


In [4]:
#Leemos el archivo reviews.parquet en un DataFrame
itemsdf = pq.read_table("../0-DATA/australian_users_itemsEDA.parquet").to_pandas()
itemsdf.head(5)

,item_id,item_name,user_id,playtime_hours
0,10,Counter-Strike,76561197970982479,0.10
1,30,Day of Defeat,76561197970982479,0.12
2,300,Day of Defeat: Source,76561197970982479,78.88
3,240,Counter-Strike: Source,76561197970982479,30.88
4,3830,Psychonauts,76561197970982479,5.55


### Uniendo tablas para los ENDPOINTS

### 1

def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. Ejemplo de retorno:

In [5]:
# price, año, item_id
df_merge = pd.merge(gamesdf, reviewsdf, left_on='id', right_on='item_id', how='inner')  # Aquí puedes elegir el método de combinación (inner, outer, left, right)
df_merge

,app_name,price,id,developer,genres_clean,item_id,recommend,user_id,sentiment_analysis,año
0,Half-Life,9.99,70.0,Valve,Action,70,1,EizanAratoFujimaki,1,2015
1,Half-Life,9.99,70.0,Valve,Action,70,1,GamerFag,0,2011
2,Half-Life,9.99,70.0,Valve,Action,70,1,76561198020928326,1,2014
3,Half-Life,9.99,70.0,Valve,Action,70,1,Bluegills,2,2013
4,Half-Life,9.99,70.0,Valve,Action,70,1,76561198071955492,2,2013
...,...,...,...,...,...,...,...,...,...,...
40381,Counter-Strike: Condition Zero,9.99,80.0,Valve,Action,80,0,76561198023508728,1,2014
40382,Counter-Strike: Condition Zero,9.99,80.0,Valve,Action,80,1,Lone_walker,2,2013
40383,Counter-Strike: Condition Zero,9.99,80.0,Valve,Action,80,1,green290,1,2015
40384,Counter-Strike: Condition Zero,9.99,80.0,Valve,Action,80,1,174gamecuman700kngkakak,1,2014


In [6]:
df_desarrollador = df_merge.drop(columns=["app_name", "id", "genres_clean", "recommend", "user_id", "sentiment_analysis"])
df_desarrollador

,price,developer,item_id,año
0,9.99,Valve,70,2015
1,9.99,Valve,70,2011
2,9.99,Valve,70,2014
3,9.99,Valve,70,2013
4,9.99,Valve,70,2013
...,...,...,...,...
40381,9.99,Valve,80,2014
40382,9.99,Valve,80,2013
40383,9.99,Valve,80,2015
40384,9.99,Valve,80,2014


Guardando enpoint1

In [7]:
tabla = pa.Table.from_pandas(df_desarrollador) # convertimos el dataframe en una tabla de pyarrow
pq.write_table(tabla,"../0-DATA/1desarrollador.parquet") # guardamos la tabla en formato parquet

In [8]:
df_desarrollador.to_csv("../0-DATA/1desarrollador.csv", index=False, encoding="utf-8") 

### 2

def userdata( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

In [9]:
df_User_id = df_merge.drop(columns=["app_name", "id",	"developer", "genres_clean", "año", "sentiment_analysis"])
df_User_id

,price,item_id,recommend,user_id
0,9.99,70,1,EizanAratoFujimaki
1,9.99,70,1,GamerFag
2,9.99,70,1,76561198020928326
3,9.99,70,1,Bluegills
4,9.99,70,1,76561198071955492
...,...,...,...,...
40381,9.99,80,0,76561198023508728
40382,9.99,80,1,Lone_walker
40383,9.99,80,1,green290
40384,9.99,80,1,174gamecuman700kngkakak


Guardando enpoint2

In [10]:
tabla = pa.Table.from_pandas(df_User_id) # convertimos el dataframe en una tabla de pyarrow
pq.write_table(tabla,"../0-DATA/2User_id.parquet") # guardamos la tabla en formato parquet

In [11]:
df_User_id.to_csv("../0-DATA/2User_id.csv", index=False, encoding="utf-8") 

### 3

def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

In [12]:
# genres_clean, año, playtime_hours
df_merge2 = pd.merge(df_merge, itemsdf, on =['user_id','item_id'], how='inner')  # Aquí puedes elegir el método de combinación (inner, outer, left, right)
df_merge2

,app_name,price,id,developer,genres_clean,item_id,recommend,user_id,sentiment_analysis,año,item_name,playtime_hours
0,Half-Life,9.99,70.0,Valve,Action,70,1,EizanAratoFujimaki,1,2015,Half-Life,23.25
1,Half-Life,9.99,70.0,Valve,Action,70,1,GamerFag,0,2011,Half-Life,9.83
2,Half-Life,9.99,70.0,Valve,Action,70,1,76561198020928326,1,2014,Half-Life,93.32
3,Half-Life,9.99,70.0,Valve,Action,70,1,Bluegills,2,2013,Half-Life,1.07
4,Half-Life,9.99,70.0,Valve,Action,70,1,76561198071955492,2,2013,Half-Life,53.92
...,...,...,...,...,...,...,...,...,...,...,...,...
31047,Counter-Strike: Condition Zero,9.99,80.0,Valve,Action,80,0,76561198015050660,1,2014,Counter-Strike: Condition Zero,40.27
31048,Counter-Strike: Condition Zero,9.99,80.0,Valve,Action,80,0,76561198023508728,1,2014,Counter-Strike: Condition Zero,0.25
31049,Counter-Strike: Condition Zero,9.99,80.0,Valve,Action,80,1,green290,1,2015,Counter-Strike: Condition Zero,7.37
31050,Counter-Strike: Condition Zero,9.99,80.0,Valve,Action,80,1,174gamecuman700kngkakak,1,2014,Counter-Strike: Condition Zero,1.20


In [13]:
df_genero = df_merge2.drop(columns=["app_name", "price", "id", "developer", "item_id", "recommend", "sentiment_analysis", "item_name"])
df_genero

,genres_clean,user_id,año,playtime_hours
0,Action,EizanAratoFujimaki,2015,23.25
1,Action,GamerFag,2011,9.83
2,Action,76561198020928326,2014,93.32
3,Action,Bluegills,2013,1.07
4,Action,76561198071955492,2013,53.92
...,...,...,...,...
31047,Action,76561198015050660,2014,40.27
31048,Action,76561198023508728,2014,0.25
31049,Action,green290,2015,7.37
31050,Action,174gamecuman700kngkakak,2014,1.20


Guardando enpoint3

In [14]:
tabla = pa.Table.from_pandas(df_genero) # convertimos el dataframe en una tabla de pyarrow
pq.write_table(tabla,"../0-DATA/3genero.parquet") # guardamos la tabla en formato parquet

In [15]:
df_genero.to_csv("../0-DATA/3genero.csv", index=False, encoding="utf-8") 

### 4

def best_developer_year( año : int ): Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)

In [16]:
# developer, recommend, user_id, año, "sentiment_analysis"
df_año = df_merge.drop(columns=["app_name", "price", "id", "genres_clean", "item_id"])
df_año                       

,developer,recommend,user_id,sentiment_analysis,año
0,Valve,1,EizanAratoFujimaki,1,2015
1,Valve,1,GamerFag,0,2011
2,Valve,1,76561198020928326,1,2014
3,Valve,1,Bluegills,2,2013
4,Valve,1,76561198071955492,2,2013
...,...,...,...,...,...
40381,Valve,0,76561198023508728,1,2014
40382,Valve,1,Lone_walker,2,2013
40383,Valve,1,green290,1,2015
40384,Valve,1,174gamecuman700kngkakak,1,2014


Guardando enpoint4

In [17]:
tabla = pa.Table.from_pandas(df_año) # convertimos el dataframe en una tabla de pyarrow
pq.write_table(tabla,"../0-DATA/4año.parquet") # guardamos la tabla en formato parquet

In [18]:
df_año.to_csv("../0-DATA/4año.csv", index=False, encoding="utf-8") 

### 5

def developer_reviews_analysis( desarrolladora : str ): Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.

In [19]:
# developer, sentiment_analysis
df_desarrolladora = df_merge.drop(columns=["app_name", "price", "id", "genres_clean", "item_id", "recommend", "user_id", "año"])
df_desarrolladora                           

,developer,sentiment_analysis
0,Valve,1
1,Valve,0
2,Valve,1
3,Valve,2
4,Valve,2
...,...,...
40381,Valve,1
40382,Valve,2
40383,Valve,1
40384,Valve,1


Guardando enpoint5

In [20]:
tabla = pa.Table.from_pandas(df_desarrolladora) # convertimos el dataframe en una tabla de pyarrow
pq.write_table(tabla,"../0-DATA/5desarrolladora.parquet") # guardamos la tabla en formato parquet

In [21]:
df_desarrolladora.to_csv("../0-DATA/5desarrolladora.csv", index=False, encoding="utf-8") 

DF para modelado

In [22]:
gamesdf

,app_name,price,id,developer,genres_clean
0,Lost Summoner Kitty,4.99,761140.0,Kotoshiro,Action
1,Ironbound,0.00,643980.0,Secret Level SRL,Free to Play
2,Real Pool 3D - Poolians,0.00,670290.0,Poolians.com,Casual
3,弹炸人2222,0.99,767400.0,彼岸领域,Action
4,Battle Royale Trainer,3.99,772540.0,Trickjump Games Ltd,Action
...,...,...,...,...,...
27457,Kebab it Up!,1.99,745400.0,Bidoniera Games,Action
27458,Colony On Mars,1.99,773640.0,"Nikita ""Ghost_RUS""",Casual
27459,LOGistICAL: South Africa,4.99,733530.0,Sacada,Casual
27460,Russian Roads,1.99,610660.0,Laush Dmitriy Sergeevich,Indie


In [23]:
gamesdf.rename(columns={"id": "item_id","genres_clean":"genres"}, inplace=True)

In [24]:
gamesdf["item_id"] = gamesdf["item_id"].astype("int")


In [25]:
gamesdf

,app_name,price,item_id,developer,genres
0,Lost Summoner Kitty,4.99,761140,Kotoshiro,Action
1,Ironbound,0.00,643980,Secret Level SRL,Free to Play
2,Real Pool 3D - Poolians,0.00,670290,Poolians.com,Casual
3,弹炸人2222,0.99,767400,彼岸领域,Action
4,Battle Royale Trainer,3.99,772540,Trickjump Games Ltd,Action
...,...,...,...,...,...
27457,Kebab it Up!,1.99,745400,Bidoniera Games,Action
27458,Colony On Mars,1.99,773640,"Nikita ""Ghost_RUS""",Casual
27459,LOGistICAL: South Africa,4.99,733530,Sacada,Casual
27460,Russian Roads,1.99,610660,Laush Dmitriy Sergeevich,Indie


In [26]:
modelo_games = gamesdf[["item_id", "app_name", "genres"]]

In [27]:
modelo_games

,item_id,app_name,genres
0,761140,Lost Summoner Kitty,Action
1,643980,Ironbound,Free to Play
2,670290,Real Pool 3D - Poolians,Casual
3,767400,弹炸人2222,Action
4,772540,Battle Royale Trainer,Action
...,...,...,...
27457,745400,Kebab it Up!,Action
27458,773640,Colony On Mars,Casual
27459,733530,LOGistICAL: South Africa,Casual
27460,610660,Russian Roads,Indie


Guardando el df poara el modelado

In [28]:
tabla = pa.Table.from_pandas(modelo_games) # convertimos el dataframe en una tabla de pyarrow
pq.write_table(tabla,"../0-DATA/modelo_games.parquet") # guardamos la tabla en formato parquet

In [29]:
modelo_games.to_csv("../0-DATA/modelo_games.csv", index=False, encoding="utf-8") 